In [2]:
import pandas as pd
import numpy as np
GBQ_PROJECT_ID = '620265099307'

In [80]:
q='''
SELECT
  month,
  rx.pct,
ccg.org_type,
ccg.name,
  rx.bnf_name,
  rx.bnf_code,
  SUM(quantity) AS quantity,
  SUM(quantity*ncso.add_cost_per_qty) AS add_cost
FROM
  `hscic.normalised_prescribing_standard` AS rx
JOIN
  `dmd.ncso_calculator` AS ncso
ON
  rx.bnf_code = ncso.bnf_code
  AND rx.month=ncso.date
FULL OUTER JOIN
  hscic.ccgs AS ccg
ON
  rx.pct=ccg.code
GROUP BY
rx.pct,
ccg.org_type,
ccg.name,
  rx.bnf_code,
  rx.bnf_name,
  month,
  ncso.add_cost_per_qty
UNION ALL
SELECT
  ncso.date,
    rx.pct,
    ccg.org_type,
    ccg.name,
    rx.bnf_name,
  rx.bnf_code,
  SUM(quantity) AS quantity,
  SUM(quantity*ncso.add_cost_per_qty) AS add_cost
FROM
  `hscic.normalised_prescribing_standard_latest_month` AS rx
JOIN
  `dmd.ncso_calculator` AS ncso
ON
  rx.bnf_code = ncso.bnf_code
  JOIN
  hscic.ccgs AS ccg
ON
  rx.pct=ccg.code
WHERE
  ncso.date>(
  SELECT
    MAX(rx1.month)
  FROM
    hscic.normalised_prescribing_standard_latest_month AS rx1)
  GROUP BY
    rx.pct,
    ccg.org_type,
    ccg.name,
    rx.bnf_code,
    rx.bnf_name,
    ncso.date,
    ncso.add_cost_per_qty'''
   

   
    
all_ncso = pd.read_gbq(q, GBQ_PROJECT_ID, verbose=False, dialect='standard')
all_ncso
###ccg_ncso = all_ncso.loc[ccg.org_type =='ccg']##
###nonccg_ncso  = all_ncso.loc[ccg.org_type !='ccg']##

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas_gbq\gbq.py:556: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  "verbosity", FutureWarning, stacklevel=1)


,month,pct,org_type,name,bnf_name,bnf_code,quantity,add_cost
0,NaT,None,PCT,Sunderland Teaching,None,None,None,nan
1,NaT,None,PCT,Enfield,None,None,None,nan
2,NaT,None,PCT,North Staffordshire,None,None,None,nan
3,NaT,None,Unknown,THAMES VALLEY COMMISSIONING HUB,None,None,None,nan
4,NaT,None,Unknown,SOUTH WEST SOUTH COMMISSIONING HUB,None,None,None,nan
5,NaT,None,PCT,South Staffordshire,None,None,None,nan
6,NaT,None,PCT,Warwickshire,None,None,None,nan
7,NaT,None,PCT,East Sussex Downs and Weald,None,None,None,nan
8,NaT,None,Unknown,WEST YORKSHIRE COMMISSIONING HUB,None,None,None,nan
9,NaT,None,PCT,Harrow,None,None,None,nan


In [111]:
##### nonccg file
nonccg_ncso  = all_ncso.loc[all_ncso.org_type !='CCG']
nonccg_ncso.head()

,month,pct,org_type,name,bnf_name,bnf_code,quantity,add_cost
0,NaT,None,PCT,Sunderland Teaching,None,None,None,nan
1,NaT,None,PCT,Enfield,None,None,None,nan
2,NaT,None,PCT,North Staffordshire,None,None,None,nan
3,NaT,None,Unknown,THAMES VALLEY COMMISSIONING HUB,None,None,None,nan
4,NaT,None,Unknown,SOUTH WEST SOUTH COMMISSIONING HUB,None,None,None,nan


In [112]:
##### ccg file
ccg_ncso  = all_ncso.loc[all_ncso.org_type =='CCG']
ccg_ncso.head()

,month,pct,org_type,name,bnf_name,bnf_code,quantity,add_cost
18,NaT,None,CCG,NHS SOLIHULL CCG,None,None,None,nan
26,NaT,None,CCG,NHS NORTH MANCHESTER CCG,None,None,None,nan
46,NaT,None,CCG,NHS AYLESBURY VALE CCG,None,None,None,nan
63,NaT,None,CCG,NHS SOUTH READING CCG,None,None,None,nan
77,NaT,None,CCG,"NHS WINDSOR, ASCOT AND MAIDENHEAD CCG",None,None,None,nan


### 2018 Impact - all months, all orgs, all meds

In [113]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [114]:
nonccg_ncso_thisyear = nonccg_ncso.loc[(nonccg_ncso["month"]>="2018-01-01")]
nonccg_ncso_thisyear.head()

,month,pct,org_type,name,bnf_name,bnf_code,quantity,add_cost
2692,2018-08-01,RY4,Unknown,None,Propranolol HCl_Tab 40mg,0204000R0AAAJAJ,28,1.62
2698,2018-08-01,RTV,Unknown,None,Risperidone_Tab 1mg,040201030AAAAAA,204,34.07
2731,2018-08-01,RMC,Unknown,None,Oxybutynin HCl_Tab 5mg,0704020J0AAACAC,56,2.25
2761,2018-08-01,NDA,Unknown,None,Tadalafil_Tab 5mg,0704050R0AAADAD,588,458.64
2786,2018-08-01,RXA,Unknown,None,Risperidone_Tab 1mg,040201030AAAAAA,56,9.35


In [124]:
ccg_ncso_thisyear = ccg_ncso.loc[(ccg_ncso["month"]>="2018-01-01")]
ccg_ncso_thisyear.head()

,month,pct,org_type,name,bnf_name,bnf_code,quantity,add_cost
2687,2018-08-01,01J,CCG,NHS KNOWSLEY CCG,Ramipril_Tab 1.25mg,0205051R0AAAKAK,2751,23.58
2688,2018-08-01,01Y,CCG,NHS TAMESIDE AND GLOSSOP CCG,Lamotrigine_Tab 25mg,0408010H0AAACAC,10335,852.64
2689,2018-08-01,04G,CCG,NHS NENE CCG,Hydroxocobalamin_Inj 1mg/ml 1ml Amp,0901020N0AAABAB,2960,1053.76
2690,2018-08-01,00V,CCG,NHS BURY CCG,Mometasone Fur_Crm 0.1%,1304000Y0AAAAAA,2490,49.05
2691,2018-08-01,06Y,CCG,NHS SOUTH NORFOLK CCG,Amisulpride_Tab 200mg,0402010A0AAAAAA,960,129.12


### 2018 - all orgs

In [127]:
nonccg_ncso_thisyear.sum()

add_cost   6167227.26
dtype: float64

In [128]:
ccg_ncso_thisyear.sum()

add_cost   169346146.13
dtype: float64

### 2018 - all meds

In [129]:
### nonccg orgs grouped by meds
nonccg_ncso_thisyear.groupby('bnf_name').sum()

,add_cost
bnf_name,
Allopurinol_Tab 100mg,316.76
Allopurinol_Tab 300mg,433.57
Amiloride HCl_Tab 5mg,151.35
Amisulpride_Tab 100mg,291.65
Amisulpride_Tab 200mg,662.82
Amisulpride_Tab 50mg,219.01
Amitriptyline HCl_Tab 50mg,18.06
Amlodipine_Tab 10mg,212.53
Amlodipine_Tab 5mg,311.85


In [130]:
###ccg orgs grouped by meds
ccg_ncso_thisyear.groupby('bnf_name').sum()

,add_cost
bnf_name,
Allopurinol_Tab 100mg,2880883.41
Allopurinol_Tab 300mg,4279247.96
Amiloride HCl_Tab 5mg,254428.08
Amisulpride_Tab 100mg,211562.96
Amisulpride_Tab 200mg,588444.97
Amisulpride_Tab 50mg,113142.48
Amitriptyline HCl_Tab 50mg,93302.16
Amlodipine_Tab 10mg,1038900.89
Amlodipine_Tab 5mg,1547104.55


### month


In [131]:
nonccg_ncso_thisyear.groupby('month').sum()

,add_cost
month,
2018-01-01,13283.59
2018-02-01,21571.52
2018-03-01,25528.61
2018-04-01,47863.06
2018-05-01,938212.49
2018-06-01,608535.03
2018-07-01,675600.52
2018-08-01,518043.53
2018-09-01,796207.75


In [132]:
ccg_ncso_thisyear.groupby('month').sum() 

,add_cost
month,
2018-01-01,16255685.14
2018-02-01,9768310.09
2018-03-01,8894242.02
2018-04-01,4319452.42
2018-05-01,9097091.12
2018-06-01,9990264.72
2018-07-01,11426196.34
2018-08-01,9774070.28
2018-09-01,9787681.37


In [222]:
total_impact = nonccg_ncso_thisyear.groupby('month').sum() + ccg_ncso_thisyear.groupby('month').sum() 
total_impact


,add_cost
month,
2018-01-01,16268968.73
2018-02-01,9789881.61
2018-03-01,8919770.63
2018-04-01,4367315.48
2018-05-01,10035303.60
2018-06-01,10598799.75
2018-07-01,12101796.87
2018-08-01,10292113.82
2018-09-01,10583889.11


In [223]:
####figures calculated by DHSC from PCA datset and figures from OP live site
df1 = pd.read_csv('ncso.calculator.DHSC.analysis.csv')
df1

,month,DHSC,OP
0,01/01/2018,15970629.00,15085276
1,01/02/2018,9583323.00,9064992
2,01/03/2018,8818888.00,8253857
3,01/04/2018,4369089.00,4008452
4,01/05/2018,10478057.00,8442101
5,01/06/2018,10710781.00,9270966
6,01/07/2018,12417455.00,10603510
7,01/08/2018,10563113.00,9070337
8,01/09/2018,11035388.00,9082968
9,01/10/2018,nan,16759101


In [240]:
df1.set_index(pd.to_datetime(df1['month'],dayfirst=True), inplace=True)
df1

,month,DHSC,OP
month,,,
2018-01-01,01/01/2018,15970629.00,15085276
2018-02-01,01/02/2018,9583323.00,9064992
2018-03-01,01/03/2018,8818888.00,8253857
2018-04-01,01/04/2018,4369089.00,4008452
2018-05-01,01/05/2018,10478057.00,8442101
2018-06-01,01/06/2018,10710781.00,9270966
2018-07-01,01/07/2018,12417455.00,10603510
2018-08-01,01/08/2018,10563113.00,9070337
2018-09-01,01/09/2018,11035388.00,9082968


In [241]:
comparison = df1.merge(total_impact, left_index=True, right_index=True) 
comparison


,month,DHSC,OP,add_cost
month,,,,
2018-01-01,01/01/2018,15970629.00,15085276,16268968.73
2018-02-01,01/02/2018,9583323.00,9064992,9789881.61
2018-03-01,01/03/2018,8818888.00,8253857,8919770.63
2018-04-01,01/04/2018,4369089.00,4008452,4367315.48
2018-05-01,01/05/2018,10478057.00,8442101,10035303.60
2018-06-01,01/06/2018,10710781.00,9270966,10598799.75
2018-07-01,01/07/2018,12417455.00,10603510,12101796.87
2018-08-01,01/08/2018,10563113.00,9070337,10292113.82
2018-09-01,01/09/2018,11035388.00,9082968,10583889.11


#### Comparison with DHSC

In [249]:
pc = comparison.copy()

pc["percentagediff"] = 100*((pc.DHSC-pc.add_cost)/pc.add_cost).fillna(0)
pc

,month,DHSC,OP,add_cost,percentagediff
month,,,,,
2018-01-01,01/01/2018,15970629.00,15085276,16268968.73,-1.83
2018-02-01,01/02/2018,9583323.00,9064992,9789881.61,-2.11
2018-03-01,01/03/2018,8818888.00,8253857,8919770.63,-1.13
2018-04-01,01/04/2018,4369089.00,4008452,4367315.48,0.04
2018-05-01,01/05/2018,10478057.00,8442101,10035303.60,4.41
2018-06-01,01/06/2018,10710781.00,9270966,10598799.75,1.06
2018-07-01,01/07/2018,12417455.00,10603510,12101796.87,2.61
2018-08-01,01/08/2018,10563113.00,9070337,10292113.82,2.63
2018-09-01,01/09/2018,11035388.00,9082968,10583889.11,4.27


In [250]:
pc.drop(columns=['OP'])

,month,DHSC,add_cost,percentagediff
month,,,,
2018-01-01,01/01/2018,15970629.00,16268968.73,-1.83
2018-02-01,01/02/2018,9583323.00,9789881.61,-2.11
2018-03-01,01/03/2018,8818888.00,8919770.63,-1.13
2018-04-01,01/04/2018,4369089.00,4367315.48,0.04
2018-05-01,01/05/2018,10478057.00,10035303.60,4.41
2018-06-01,01/06/2018,10710781.00,10598799.75,1.06
2018-07-01,01/07/2018,12417455.00,12101796.87,2.61
2018-08-01,01/08/2018,10563113.00,10292113.82,2.63
2018-09-01,01/09/2018,11035388.00,10583889.11,4.27


### Non-CCG Codes

In [251]:
nonccg_ncso_thisyear.groupby('bnf_name').sum().sort_values(by='add_cost', ascending=False)

,add_cost
bnf_name,
Buprenorphine_Tab Subling 8mg S/F,3982742.62
Buprenorphine_Tab Subling 2mg S/F,1995799.31
Phenoxymethylpenicillin_Soln 125mg/5ml,20080.54
Diamorph HCl_Inj 30mg Amp,11734.47
Naproxen_Tab 500mg,11448.29
Phenoxymethylpenicillin_Soln 250mg/5ml,9891.36
Risperidone_Tab 500mcg,8550.82
Metronidazole_Tab 400mg,7059.95
Diamorph HCl_Inj 10mg Amp,6325.73


In [266]:
nonccg_ncso_thisyear.groupby('org_type').sum().sort_values(by='add_cost', ascending=False)

,add_cost
org_type,
Unknown,6167227.26


In [267]:
nonccg_ncso_thisyear.groupby('name').sum().sort_values(by='add_cost', ascending=False)

,add_cost
name,
YORKSHIRE AND HUMBER COMMISSIONING HUB,461.75
SOUTH WEST COMMISSIONING HUB,92.96
NORTH MIDLANDS COMMISSIONING HUB,79.98
SOUTH EAST COMMISSIONING HUB,40.57
SOUTH CENTRAL COMMISSIONING HUB,22.21
LONDON COMMISSIONING HUB,3.79
LANCASHIRE COMMISSIONING HUB,3.25
EAST COMMISSIONING HUB,2.44


In [268]:
nonccg_ncso_thisyear.groupby('pct').sum().sort_values(by='add_cost', ascending=False)

,add_cost
pct,
NMS,2261234.68
NKI,695935.35
NI3,241041.29
212,195113.31
RX4,190442.24
112,175160.76
NL1,138642.91
107,134666.31
114,125355.08
